In [1]:
### Install Qiskit, if needed

%pip install qiskit[visualization]==1.0.2
%pip install qiskit_aer
%pip install qiskit_ibm_runtime
%pip install matplotlib
%pip install pylatexenc
%pip install qiskit-transpiler-service
%pip install git+https://github.com/qiskit-community/Quantum-Challenge-Grader.git

  Using cached qiskit-1.0.2-cp38-abi3-win_amd64.whl.metadata (12 kB)
Using cached qiskit-1.0.2-cp38-abi3-win_amd64.whl (4.1 MB)
  Attempting uninstall: qiskit
    Found existing installation: qiskit 1.2.4
    Uninstalling qiskit-1.2.4:
      Successfully uninstalled qiskit-1.2.4
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
qiskit-aer 0.15.1 requires qiskit>=1.1.0, but you have qiskit 1.0.2 which is incompatible.
qiskit-ibm-runtime 0.28.0 requires qiskit>=1.1.0, but you have qiskit 1.0.2 which is incompatible.


  Using cached qiskit-1.2.4-cp38-abi3-win_amd64.whl.metadata (13 kB)
Using cached qiskit-1.2.4-cp38-abi3-win_amd64.whl (4.6 MB)
  Attempting uninstall: qiskit
    Found existing installation: qiskit 1.0.2
    Uninstalling qiskit-1.0.2:
      Successfully uninstalled qiskit-1.0.2
Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


^C
Note: you may need to restart the kernel to use updated packages.


In [2]:
# imports ()
import random
import numpy as np
from typing import List, Callable
from scipy.optimize import minimize
from scipy.optimize._optimize import OptimizeResult
import matplotlib.pyplot as plt

from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector, Operator, SparsePauliOp
from qiskit.primitives import StatevectorSampler, PrimitiveJob
from qiskit.circuit.library import TwoLocal, MCXGate, MCMT, ZGate
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.visualization import plot_histogram
from qiskit_ibm_runtime.fake_provider import FakeSherbrooke
from qiskit_ibm_runtime import Session, EstimatorV2 as Estimator
from qiskit_aer import AerSimulator

import random
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator, Aer
from qiskit.visualization import plot_histogram
from qiskit.circuit.library import ZGate


In [3]:
def check_accuracy(num_trials=10):
    results = []
    for _ in range(num_trials):
        main(results)
        
    count = 0
    for res in results:
        if res:
            count += 1
    
        
    print('Accuracy: {}'.format(count / len(results)))

In [4]:
def set_io_qubits(qubit_count):
    """Initializes a quantum circuit with input/output qubits."""
    return QuantumCircuit(qubit_count + 1, qubit_count)

def make_oracle(circuit, qubit_count, x_bits):
    """Creates an oracle that flips the state of the qubits matching x_bits."""
    for idx, bit in enumerate(x_bits):
        if bit == 0:
            circuit.x(idx)
    # Apply multi-controlled Z gate (oracle)
    circuit.h(qubit_count)
    circuit.mcx(list(range(qubit_count)), qubit_count)  # Multi-controlled NOT
    circuit.h(qubit_count)
    for idx, bit in enumerate(x_bits):
        if bit == 0:
            circuit.x(idx)
    circuit.barrier()

def make_grover_circuit(qubit_count, x_bits):
    """Find the value recognized by the oracle in sqrt(N) attempts."""
    # Initialize circuit
    circuit = set_io_qubits(qubit_count)
    
    # Initialize qubits
    circuit.h(range(qubit_count))  # Apply Hadamard to the main qubits
    circuit.x(qubit_count)         # Ancilla qubit set to |1⟩ for controlled operations
    
    # Add oracle
    make_oracle(circuit, qubit_count, x_bits)
    
    # Grover operator (Diffusion operator)
    circuit.h(range(qubit_count))
    circuit.x(range(qubit_count))
    
    circuit.h(qubit_count - 1)
    circuit.mcx(list(range(qubit_count - 1)), qubit_count - 1)  # Multi-controlled Z gate
    circuit.h(qubit_count - 1)
    
    circuit.x(range(qubit_count))
    circuit.h(range(qubit_count))
    
    # Measure
    circuit.measure(range(qubit_count), range(qubit_count))
    
    return circuit

In [5]:
def main(solution=False, qubit_count=4, shots=1024, results=[], output=False):
    for _ in range(3):
        if not solution:
            # Choose the x' and make an oracle which can recognize it.
            x_bits = [random.randint(0, 1) for _ in range(qubit_count)]
        else:
            x_bits = solution.pop(0)  # Use one of the predefined solutions
            
        rev_x_bits = x_bits[::-1]

        # Make Grover's circuit
        circuit = make_grover_circuit(qubit_count, rev_x_bits)

        # Simulate the circuit
        simulator = Aer.get_backend('qasm_simulator')
        compiled_circuit = transpile(circuit, simulator, optimization_level=3)
        result = simulator.run(compiled_circuit, shots=shots).result()
        counts = result.get_counts()

        # Check if we actually found the secret value
        most_common_bitstring = max(counts, key=counts.get)

        # prints information
        if output:
            print('Secret bit sequence: {}'.format(x_bits))
            print('Circuit:')
            print(circuit)
            print('Sampled results:\n{}'.format(counts))
            print('Most common bitstring: {}'.format(most_common_bitstring))
            print('Found a match: {}'.format(
                most_common_bitstring == ''.join(map(str, x_bits))))
        else:
            results.append(most_common_bitstring == ''.join(map(str, x_bits)))

In [6]:
if __name__ == '__main__':
    
    main(solution=[[0,0,1,1], [0, 1, 1, 1], [1,0,0,1]], output=True)

Secret bit sequence: [0, 0, 1, 1]
Circuit:
     ┌───┐                ░ ┌───┐┌───┐          ┌───┐┌───┐     ┌─┐         
q_0: ┤ H ├───────■────────░─┤ H ├┤ X ├───────■──┤ X ├┤ H ├─────┤M├─────────
     ├───┤       │        ░ ├───┤├───┤       │  ├───┤├───┤     └╥┘┌─┐      
q_1: ┤ H ├───────■────────░─┤ H ├┤ X ├───────■──┤ X ├┤ H ├──────╫─┤M├──────
     ├───┤┌───┐  │  ┌───┐ ░ ├───┤├───┤       │  ├───┤├───┤      ║ └╥┘┌─┐   
q_2: ┤ H ├┤ X ├──■──┤ X ├─░─┤ H ├┤ X ├───────■──┤ X ├┤ H ├──────╫──╫─┤M├───
     ├───┤├───┤  │  ├───┤ ░ ├───┤├───┤┌───┐┌─┴─┐├───┤├───┤┌───┐ ║  ║ └╥┘┌─┐
q_3: ┤ H ├┤ X ├──■──┤ X ├─░─┤ H ├┤ X ├┤ H ├┤ X ├┤ H ├┤ X ├┤ H ├─╫──╫──╫─┤M├
     ├───┤├───┤┌─┴─┐├───┤ ░ └───┘└───┘└───┘└───┘└───┘└───┘└───┘ ║  ║  ║ └╥┘
q_4: ┤ X ├┤ H ├┤ X ├┤ H ├─░─────────────────────────────────────╫──╫──╫──╫─
     └───┘└───┘└───┘└───┘ ░                                     ║  ║  ║  ║ 
c: 4/═══════════════════════════════════════════════════════════╩══╩══╩══╩═
                                             

In [22]:
for i in range(10):
    main(output=True)
    print('\n')

Secret bit sequence: [1, 0, 0]
Circuit:
     ┌───┐┌───┐   ┌───┐┌───┐┌───┐   ┌───┐┌───┐┌─┐      
q_0: ┤ H ├┤ X ├─■─┤ X ├┤ H ├┤ X ├─■─┤ X ├┤ H ├┤M├──────
     ├───┤├───┤ │ ├───┤├───┤├───┤ │ ├───┤├───┤└╥┘┌─┐   
q_1: ┤ H ├┤ X ├─■─┤ X ├┤ H ├┤ X ├─■─┤ X ├┤ H ├─╫─┤M├───
     ├───┤└───┘ │ ├───┤├───┤└───┘ │ ├───┤├───┤ ║ └╥┘┌─┐
q_2: ┤ H ├──────■─┤ H ├┤ X ├──────■─┤ X ├┤ H ├─╫──╫─┤M├
     ├───┤        └───┘└───┘        └───┘└───┘ ║  ║ └╥┘
q_3: ┤ X ├─────────────────────────────────────╫──╫──╫─
     └───┘                                     ║  ║  ║ 
c: 3/══════════════════════════════════════════╩══╩══╩═
                                               0  1  2 
Sampled results:
{'100': 694, '000': 49, '011': 46, '110': 62, '010': 34, '111': 43, '101': 46, '001': 50}
Most common bitstring: 100
Found a match: True


Secret bit sequence: [1, 1, 0]
Circuit:
     ┌───┐┌───┐   ┌───┐┌───┐┌───┐   ┌───┐┌───┐┌─┐      
q_0: ┤ H ├┤ X ├─■─┤ X ├┤ H ├┤ X ├─■─┤ X ├┤ H ├┤M├──────
     ├───┤└───┘ │ ├───┤├───┤└───┘ │ 

In [ ]:
check_accuracy(100)